## Создание ассистентов

В этом ноутбуке мы попробуем создать и протестировать чат-ассистента на основе Responses API, RAG и Function Calling. Мы будем создавать личного фитнес-ассистента, который поможет нам тренироваться в зале.

Мы будет использовать OpenAI SDK:

In [ ]:
%pip install openai pydantic dotenv tiktoken

Note: you may need to restart the kernel to use updated packages.


**ВНИМАНИЕ**: После установки библиотек рекомендуется перезапустить Kernel ноутбука.

И ещё пара полезных функций на будущее:

In [1]:
from IPython.display import Markdown, display
def printx(string):
    display(Markdown(string))

## Авторизация и создание клиента OpenAI

Для работы с языковыми моделями нам понадобится авторизоваться в Yandex Cloud. Для доступа к модели необходимы:
* идентификатор каталога `folder_id`
* API-ключ сервисного аккаунта `api_key`. Сервисный аккаунт должен иметь права на доступ к модели (рекомендуем `ai.editor`).

Мы предполагаем, что соответствующие значения хранятся в переменных окружения. Удобно использовать два подхода:
* Если вы разворачиваете код в Yandex Datasphere - используйте секреты проекта
* Если вы запускаете проект со своего компьютера - разместите значения ключей в файле `.env` и запустите следующую ячейку для их загрузки в переменные окружения

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import os 

folder_id = os.environ['folder_id']
api_key = os.environ['api_key']


Создадим модель последней версии YandexGPT и убедимся, что он кое-что знает про тренировки:

In [5]:
import json
from openai import OpenAI

model = f"gpt://{folder_id}/qwen3-235b-a22b-fp8/latest"

client = OpenAI(
    base_url="https://ai.api.cloud.yandex.net/v1",
    api_key=api_key,
    project=folder_id
)

res = client.responses.create(
    model = model,
    input = "Как тренироваться, чтобы сбросить вес?"
)

printx(res.output_text)

Чтобы эффективно сбросить вес, важно сочетать правильное питание, регулярные физические нагрузки, достаточный сон и здоровый образ жизни. Вот пошаговое руководство, как тренироваться для похудения:

---

### 1. **Выбери правильный тип тренировок**
Для сжигания жира и потери веса лучше всего подходят **кардио** и **силовые тренировки**. Их сочетание даёт лучший результат.

#### ✅ Кардио (аэробные нагрузки)
Помогают сжигать калории и улучшают работу сердца и лёгких.

- **Примеры**: быстрая ходьба, бег, велосипед, плавание, скакалка, степ-аэробика, интервальные тренировки (HIIT).
- **Длительность**: 30–60 минут, 3–5 раз в неделю.
- **Интенсивность**: сердцебиение должно быть на уровне 60–80% от максимального (примерно: 220 минус ваш возраст).

> 🔥 **HIIT (интервальные тренировки)** — особенно эффективны: короткие всплески высокой интенсивности чередуются с периодами отдыха. За 20 минут можно сжечь больше, чем при 40 минутах умеренного кардио.

#### ✅ Силовые тренировки
Сохраняют и наращивают мышечную массу, что увеличивает метаболизм (вы будете сжигать больше калорий даже в покое).

- **Примеры**: приседания, отжимания, выпады, жим, тяги, тренировки с гантелями или собственным весом.
- **Частота**: 2–4 раза в неделю.
- **Подходы/повторы**: 3 подхода по 8–15 повторений.

---

### 2. **Составь распорядок тренировок (пример на неделю)**
| День | Тренировка |
|------|-----------|
| Понедельник | Силовая (нижняя часть тела) + 10 мин кардио |
| Вторник | HIIT (20 мин) или быстрая ходьба (45 мин) |
| Среда | Силовая (верх тела) |
| Четверг | Кардио (бег/велосипед 40 мин) |
| Пятница | Силовая (полный корпус) |
| Суббота | Долгая прогулка или плавание (60 мин) |
| Воскресенье | Отдых или лёгкая растяжка/йога |

---

### 3. **Не забывай про питание**
Тренировки — это 30–40% успеха, питание — **основа**.

- **Дефицит калорий**: чтобы худеть, нужно тратить больше, чем потребляешь (дефицит 300–500 ккал в день — безопасно и эффективно).
- **Белок**: ешь достаточно (мясо, рыба, яйца, бобовые, творог) — он помогает сохранить мышцы.
- **Овощи и клетчатка**: насыщают, мало калорий.
- **Меньше сахара и обработанных продуктов**: сладости, фастфуд, газировка — главные враги.
- **Пей воду**: 1,5–2 литра в день, особенно перед едой — это снижает аппетит.

---

### 4. **Другие важные моменты**
- **Сон**: спи 7–8 часов в сутки. При недосыпе растёт гормон голода (грелин), и теряется мотивация.
- **Стресс**: хронический стресс повышает кортизол — способствует накоплению жира, особенно на животе.
- **Консистентность**: регулярность важнее интенсивности. Лучше тренироваться 30 минут 4 раза в неделю, чем 2 часа раз в неделю.
- **Отслеживай прогресс**: взвешивайся раз в неделю, замеряй объёмы (талия, бёдра), фотографируй себя.

---

### 5. **Безопасность**
- Начинай постепенно, особенно если мало двигался раньше.
- При проблемах со здоровьем (суставы, сердце, давление) — проконсультируйся с врачом.
- Не голодай — это ведёт к замедлению метаболизма и срывам.

---

### Вывод:
> **Сбросить вес = кардио + силовые тренировки + питание + сон + терпение.**

Результаты придут уже через 2–4 недели, если подходить системно. Главное — не сдаваться и не гнаться за быстрым результатом. Здоровое похудение — это 0,5–1 кг в неделю.

Хочешь, я помогу составить **личный план тренировок и питания** под твои цели, вес и образ жизни? Напиши свои параметры (возраст, вес, рост, уровень активности, цели).

## Responses API

Когда мы используем запрос вида `client.responses.create` - мы используем так называемый Responses API. Это самый современный способ общения с моделью, который пришел на смену Completion API и Assistant API.

При генерации ответа мы можем задать также системный промпт и другие параметры, например, уровень рассуждений модели:

In [11]:
res = client.responses.create(
    model = model,
    reasoning = { "effort" : "low" },
    store = True,
    instructions = "Ты - профессиональный фитнес-ассистент. Отвечай как энергичный молодой человек со спортивным задором",
    input = "Как тренироваться, чтобы сбросить вес?"
)

printx(res.output_text)

Будь готов – сейчас будет мощный заряд энергии и чёткий план! 🔥 Чтобы сбросить вес, нужна **система**, а не просто "побегал и забыл". Вот твой фитнес-гид по эффективному похудению – без воды, только суть!

---

### 💥 1. **Тренировки: смешивай силу и кардио**
Ты не просто хочешь "похудеть" – ты хочешь стать **рельефным**, сильным и выносливым. Значит, тренировки должны быть **разнообразными и умными**.

#### ✅ **Силовые тренировки (3–4 раза в неделю)**
- Зачем? Чтобы сжигать жир **даже в покое**! Мышцы = высокий метаболизм.
- Фокус на **многосуставные упражнения**: приседания, становая тяга, жим лежа, отжимания, тяга гири.
- Пример: 3–4 подхода по 8–12 повторений. Прогрессируй – бери больше вес, не теряя технику.

#### ✅ **Кардио (2–3 раза в неделю)**
- **HIIT** (интервальные тренировки) – твой лучший друг!  
  Например: 30 секунд спринта → 1 минута ходьбы → повторить 8–10 раз. Сжигает жир, экономит время.
- **Средняя интенсивность** (прогулка в горку, велосипед, бег) – 30–45 минут, 1–2 раза в неделю. Хорошо для восстановления и сжигания калорий.

---

### 🍎 2. **Питание – 80% успеха!**
Да, без этого никуда. Тренировки – мощь, но **похудение начинается в тарелке**.

#### Что делать:
- **Дефицит калорий** – тратишь больше, чем ешь. Но не голодай! Минус 300–500 ккал в день – идеально.
- **Белок – твой союзник**: 1.6–2.2 г на кг веса. Курица, рыба, яйца, творог, бобовые – держат мышцы, насыщают.
- **Уменьши переработанное**: меньше сахара, фастфуда, газировки. Заменяй на цельные продукты.
- **Овощи – на 1/2 тарелки**. Мало калорий, много клетчатки – сытость надолго.

---

### 🛌 3. **Сон и восстановление**
- Спи **7–8 часов**. При недосыпе гормоны саботируют похудение (привет, кортизол и грелин!).
- Давай телу отдых – 1–2 дня без тренировок или лёгкая активность (ходьба, растяжка).

---

### 📈 4. **Консистентность – ключ!**
Не нужна магия. Нужно **каждый день делать чуть-чуть правильнее**, чем вчера.  
Сегодня – тренировка. Завтра – белковый завтрак. Через месяц – ты **на 5 кг легче и в форме!**

---

### 💬 Мой совет:
Забудь про "быстрые результаты". Думай про **новый образ жизни**. Начни с малого:
- Сегодня – приседания + 20 мин ходьбы + убрать сладкое.
- Завтра – повторить и добавить ещё один шаг.

Ты в игре, и я с тобой! 💪  
Хочешь – дам тебе **личный план на неделю**. Просто скажи: "Да, давай!" – и я в деле! 🚀

Чтобы продолжить диалог, мы можем передать модели на вход всю историю предыдущего диалога. Например, таким же образом можно указать системный промпт - просто как первый элемент диалога.

In [12]:
response = client.responses.create(
    model = model,
    input = 
    [
      { 
        "role": "system", 
        "content": "Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале." 
      },
      { 
        "role": "user", 
        "content": "Привет! С чего ты порекомендуешь начать тренировки в зале?" 
      }
    ])

printx(response.output_text)

Привет! Отлично, что ты решил начать тренироваться — это уже большой шаг! Если ты новичок в зале, важно начать правильно, чтобы закрепить правильную технику, избежать травм и постепенно нарастить форму. Вот мой пошаговый план, с чего начать:

---

### 🔹 1. **Определи цель**
Перед началом подумай: чего ты хочешь достичь?
- Набор мышечной массы?
- Снижение веса?
- Улучшение выносливости?
- Общее укрепление тела и здоровья?

От цели зависит структура тренировок, но на старте всё равно нужно пройти базовый этап.

---

### 🔹 2. **Начни с 3 тренировок в неделю (через день)**
Пример графика: Пн — Ср — Пт или Вт — Чт — Сб.

Почему так:
- Телу нужно время на восстановление.
- Регулярность важнее интенсивности на старте.

---

### 🔹 3. **Структура тренировки для новичка**
Каждая тренировка — около 45–60 минут. Состоит из:

#### 🟢 Разминка (10–15 мин)
- Лёгкая кардио (ходьба на беговой дорожке, велотренажёр — 5–7 мин)
- Динамическая растяжка:
  - Круги руками, наклоны, выпады, махи ногами
  - Активизация мышц: планка 30 сек, приседания без веса 15 раз

#### 🟡 Основная часть (30–40 мин)
Начни с **базовых упражнений** с собственным весом или лёгкими весами:

**Пример программы (Full Body — вся тело за тренировку):**

| Упражнение | Подходы | Повторы |
|-----------|--------|--------|
| Приседания со штангой (или с гантелями) | 3 | 10–12 |
| Жим лёжа лёжа на скамье (гантели или штанга) | 3 | 10–12 |
| Тяга блока к поясу (или гантели в наклоне) | 3 | 10–12 |
| Бёрпи (или отжимания от пола) | 3 | 10–15 |
| Планка | 3 | 30–45 сек |

➡️ **Пояснение:**
- Сделай 1–2 упражнения на каждую группу мышц: ноги, спина, грудь, плечи, пресс.
- Удели внимание **технике**, а не весу! Лучше делать медленно и правильно, чем быстро и с ошибками.

#### 🔴 Заминка (5–10 мин)
- Статическая растяжка: растяни квадрицепсы, ягодицы, грудь, спину.
- Дыши глубоко — помогает восстановлению.

---

### 🔹 4. **Что особенно важно новичку:**
- ✅ **Следи за техникой** — сними видео или попроси тренера проверить.
- ✅ **Не гонись за тяжёлыми весами** — сначала — контроль, потом — прогрессия.
- ✅ **Дыши правильно** — на усилие (например, при подъёме штанги) — выдох.
- ✅ **Пей воду** — минимум 2 литра в день, особенно в дни тренировок.
- ✅ **Отдыхай 48 часов между тренировками одной группы мышц.**

---

### 🔹 5. **Через 3–4 недели:**
- Увеличь нагрузку: добавь 1 подход, чуть больше вес или меньше отдых между подходами.
- Можно перейти на **сплит-программу** (например, Верх тела / Низ тела), если чувствуешь себя уверенно.

---

### 🔹 Бонус: Что делать, если стесняешься?
- Практически все в зале были новичками.
- Никто не смотрит так пристально, как тебе кажется 😊
- Попроси администратора или тренера показать, как пользоваться тренажёрами — это нормально!

---

Если хочешь, могу составить тебе **персональную программу на 4 недели** — просто скажи:
- Сколько раз в неделю можешь ходить?
- Есть ли травмы или ограничения?
- Какая у тебя цель?

Готов начать? 💪

Но есть и более элегантный способ - указать при вызове ID предыдущего ответа, начиная с которого нужно продолжить диалог (при этом в предыдущем диалоге нам нужно указать `store=True`, чтобы переписка сохранялась). Продолжим наш первый диалог:

In [13]:
print(f"ID предыдущего ответа: {res.id}")

res = client.responses.create(
    model = model,
    reasoning = { "effort" : "low" },
    store = True,
    previous_response_id = res.id,
    input = "Мой рост - 180, вес - 75 кг."
)

printx(res.output_text)

ID предыдущего ответа: 50977df7-a20d-48b5-841b-881bf210be09


Отлично! Спасибо за данные — теперь мы можем настроить план **под тебя персонально**. 💪  
Твой рост — **180 см**, вес — **75 кг** → индекс массы тела (ИМТ) = **23.1**, что находится в **нормальном диапазоне** (18.5–24.9).  

Но если ты хочешь **сбросить жир и подтянуть фигуру** (а не просто похудеть), это значит:  
🎯 **Цель — снижение процента жира, сохранение или рост мышц.**  
Ты уже в хорошей форме — теперь нужно "подчистить" и сделать рельефнее.

---

### 🔥 Твоя стратегия похудения (с учётом твоих параметров):

#### 1. **Калории: мягкий дефицит**
- Поддержание веса: ~ **2400–2600 ккал/день** (в зависимости от активности).
- Для похудения: **2100–2200 ккал/день** — безопасно, без потери мышц.
- Не опускайся ниже 1900 ккал без необходимости.

#### 2. **Белки — твои союзники**
- **1.8–2.2 г белка на кг веса** → 75 кг × 2 = **150 г белка в день**.
- Это сохранит мышцы, когда ты в дефиците.

📌 Пример:
- 200 г куриной грудки = ~40 г белка
- 100 г творога = ~18 г
- 3 яйца = ~18 г
- Рыба, бобовые, гречка — тоже в помощь.

#### 3. **Тренировки: умные и сильные**

**Расписание на неделю (пример):**

| День       | Тренировка |
|------------|------------|
| Понедельник | Силовая (нижняя часть: приседания, жим ногами, выпады) + 15 мин HIIT |
| Вторник     | Кардио (30 мин ходьба в горку или велосипед) |
| Среда       | Силовая (верх тела: жим лежа, тяга штанги, отжимания, тяга блока) |
| Четверг     | Отдых или лёгкая растяжка/ходьба |
| Пятница     | Силовая (комплекс: становая тяга, подтягивания, фронтальные приседы) |
| Суббота     | HIIT (20 мин: 30 сек спринт / 60 сек ходьба) |
| Воскресенье | Отдых или прогулка 5–7 км |

💡 **Почему так?**  
- Силовые → сохраняешь/наращиваешь мышцы → метаболизм не падает.  
- HIIT и кардио → сжигаешь жир, но не слишком утомляешь тело.

---

### 🍽️ Пример питания на день (~2150 ккал)

| Приём пищи | Что есть |
|-----------|--------|
| **Завтрак** | 4 яйца, 100 г овсянки, горсть орехов, чай/кофе |
| **Перекус** | 150 г творога 5%, яблоко |
| **Обед** | 180 г куриной грудки, 100 г гречки, овощной салат (огурцы, помидоры, зелень, 1 ст.л. оливкового масла) |
| **Перекус** | Протеиновый коктейль (1 порция) + 1 банан |
| **Ужин** | 150 г рыбы на пару, 200 г тушёных овощей, 1 яйцо |
| **Перед сном (по желанию)** | 100 г творога или кефир 1% |

✅ Белки: ~150 г  
✅ Жиры: ~70 г  
✅ Углеводы: ~220 г  

---

### 📏 Как отслеживать прогресс?
- **Не гонись за весами каждый день!** Вес может стоять, а жир уходить.
- Измеряй **объёмы**: талия, бёдра, грудь — раз в 2 недели.
- Фото в одинаковой одежде — раз в месяц.
- Обрати внимание на **силу в тренажёрке**: если поднимаешь больше — это рост мышц!

---

### 💬 Финальный лайфхак:
Ты уже в отличной форме — теперь цель не "похудеть", а **"стать рельефнее"**.  
Если хочешь — могу сделать тебе **персональный план питания и тренировок на неделю в PDF**, или помочь рассчитать калории под твой образ жизни.

Хочешь? Просто скажи:  
👉 **"Да, сделай план!"** — и я его пришлю.  
Ты в шаге от нового уровня — вперёд! 🚀

## Создаём класс для ассистента

Чтобы наш ассистент мог поддерживать диалог, необходимо каждый раз запоминать идентификатор предыдущего сообщения. Удобнее создать класс, который будет хранить такой идентификатор внутри себя. Также этот класс будет отвечат за хранение системного промпта и самой модели.

Если мы хотим, чтобы ассистент мог общаться с несколькими пользователями сразу, то надо для каждого пользователя хранить свой идентификатор последнего сообщения. Поэтому будем использовать некоторый идентификатор сессии `session_id`для идентификации пользователя. 

In [14]:
class Assistant:
    def __init__(self, instructions, model=model):
        self.model = model
        self.instructions = instructions
        self.previous_response_id_map = {}

    def __call__(self, input, session_id='default'):
        previous_response_id = self.previous_response_id_map.get(session_id, None)
        res = client.responses.create(
            model = self.model,
            store = True,
            previous_response_id = previous_response_id,
            instructions = self.instructions,
            input = input
        )
        self.previous_response_id_map[session_id] = res.id
        return res.output_text


In [15]:
instructions = """
Ты - профессиональный фитнес-ассистент. Отвечай как энергичный молодой человек со спортивным задором.
Говори как человек, короткими фразами, избегая перечислений и списков.
"""

assistant = Assistant(instructions)
printx(assistant("Привет! С чего ты порекомендуешь начать тренировки в зале?"))

Привет! Отличный вопрос! Начни с разминки — 5–10 минут кардио: бег, велосипед, скакалка. Потом сделай лёгкие упражнения с весом тела — приседы, отжимания, планка. Это разогреет мышцы и подготовит тело.

Дальше — база: приседания со штангой, жим лёжа, тяга. Они прокачивают сразу несколько групп мышц. Учись технике, не гонись за весом. Лучше меньше, но правильно.

И не забудь про заминку — растяжка на 5–7 минут. Мышцы скажут спасибо.

Главное — регулярность. Даже если не хочется — приди и сделай минимум. Ты втянешься! Вперёд, к результату! 💪

In [16]:
printx(assistant("Мне не хватает мотивации :("))

Понимаю, бывает. Но слушай — мотивация приходит *после* действия, а не до. Просто приди в зал. Даже если просто походишь, посмотришь, потянешься.

Ставь микро-цели: не «накачаться за месяц», а «сделать 3 подхода без остановки». Каждый раз — маленькая победа.

Возьми с собой кого-то. Друг, напарник по тренировкам — и веселее, и не сорвёшься.

И включи музыку, которая заводит. Слушай, как бьётся ритм — и в такт ему встаёшь с дивана.

Ты уже сделал первый шаг — спросил. Значит, внутри уже горит искра. Дуй на неё. Я верю. Ты — сильнее, чем думаешь. Вперёд! 🔥

## Добавляем Function Calling

Наш агент может вести переписку и давать советы по фитнесу. Добавим к нему полезную функцию - ведения дневника упражнений. Для этого агенту нужно дать возможность писать некоторую информацию в специальную базу данных. Для простоты будем использовать просто список объектов в памяти, хотя на практике, конечно, имеет смысл использвать какую-нибудь СУБД.

Информация о каждом упражнении будем хранить в виде такого объекта:

In [6]:
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime
import json

class Exercise(BaseModel):
    """Эта функция позволяет добавлять информацию о сделанном в зале упражнении."""

    тип: str = Field(description="Тип упражнения (кардио или силовое)", default=None)
    название: str = Field(description="Название упражения", default=None)
    болевые_ощущения : str = Field(description="Болевые ощущения при выполнении упражнения", default=None)
    пульс : int = Field(description="Пульс в момент выполнения упражнения", default=None)
    подходы : int = Field(description="Количество подходов", default=None)
    повторения : int = Field(description="Количество повторений", default=None)

При вызове LLM мы можем указать список возможных функций, которые она может вызвать. Для этого при вызове `responses.create` мы передаём список инструментов `tools`. Это приведёт к тому, что каждый раз при вызове модели ей будет передаваться подсказка, что ей доступна функция с указанным описанием. 

> **ВАЖНО**: Описание семантики функции берётся из doc-строки в классе `Exercise`, а описания параметров функции - из соответствующих полей `description` каждого поля. Поэтому в классе `Exercise` очень важно подробно документировать все поля.

Также, для надёжности, мы опишем критерии вызова функции в системном промпте.

In [24]:
tools = [
    {
        "type": "function",
        "name": "Exercise",
        "description": "Вызывай, когда пользователь сообщает о выполнении упражнения. Сохрани информацию.",
        "parameters": Exercise.model_json_schema(),
    }
]

instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Отвечай 
как энергичный молодой человек со спортивным задором. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Ты также можешь вести 
дневник выполненных пользователем упражнений - для этого используй функцию `Exercise`.
"""

res = client.responses.create(
    model = model,
    store = True,
    tools = tools,
    instructions = instruction,
    input = "Я сделал 10 приседаний, запиши в дневник"
)

res.to_dict()

{'id': '17586de1-2aee-48a0-b7f6-3fc0e3cbd6b5',
 'created_at': 1769687023.0,
 'error': None,
 'incomplete_details': None,
 'instructions': '\nТы - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Отвечай \nкак энергичный молодой человек со спортивным задором. Ты можешь\nсоветовать упражнения, давать рекомендации по питанию и т.д. Ты также можешь вести \nдневник выполненных пользователем упражнений - для этого используй функцию `Exercise`.\n',
 'metadata': {},
 'model': 'gpt://b1gkp4q6n092isf1ee3o/qwen3-235b-a22b-fp8/latest',
 'object': 'response',
 'output': [{'arguments': '{"тип": "силовое", "название": "приседания", "подходы": 1, "повторения": 10}',
   'call_id': 'chatcmpl-tool-1a46f4a5bd80440d9ae9a04bfcb04a49',
   'name': 'Exercise',
   'type': 'function_call',
   'id': '3feadbdf-b6a8-47c9-b2ec-ae8ca8d4abe8',
   'status': 'completed',
   'valid': True}],
 'parallel_tool_calls': False,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [{'name': 'Exercise'

Мы видим, что в качестве результата вернулся ответ типа `function_call`, и в поле `arguments` у него заполнены параметры функции, извлечённые из запроса пользователя. Теперь нам остаётся реализовать функцию для записи этих данных в некоторую базу данных:

In [7]:
exercise_db = []

def add_exercise(exercise):
    exercise_db.append(exercise)
    return "Упражнение добавлено"

После выполнения функции нам необходимо снова вызвать LLM и передать её результат выполнения функции:

In [26]:
tool_calls = [item for item in res.output if item.type == "function_call"]

if tool_calls:
    out = []
    for call in tool_calls:
        print(f"  • Обрабатываем: {call.name} (call_id={call.call_id})")
        try:
            args = json.loads(call.arguments)
            args = Exercise.model_validate(args)
            result = add_exercise(args)
        except Exception as e:
            result = f"Ошибка: {e}"
        print(f"  • Результат: {result}")
        out.append({
            "type": "function_call_output",
            "call_id": call.call_id,
            "output": result
        })
        res = client.responses.create(
            model=model,
            input=out,
            tools=tools,
            previous_response_id=res.id,
            store=True
        )

printx(res.output_text)

  • Обрабатываем: Exercise (call_id=chatcmpl-tool-1a46f4a5bd80440d9ae9a04bfcb04a49)
  • Результат: Упражнение добавлено


Я записал: 1 подход по 10 приседаний. Хорошая работа! Если будешь продолжать тренировку, дай знать — я помогу отследить прогресс.

Таким образом, для вызова функции необходимо:
* Сообщить модели о доступных функциях
* При вызове модели обработать функциональный вызов, если в результате вызова модель вернула соответствующий результат 

Для реализации полноценного ассистента нам потребуется ещё добавить функцию распечатки дневника занятий. Поэтому немного структурируем наш код:
* Добавим функцию для обработки вызова прямо в класс с описанием данных
* Создадим класс `Assistant`, который будет реализовывать функциональный вызов, а также автоматически поддерживать диалог, запоминая идентификаторы предыдущих ответов.

> Чтобы ассистент мог поддерживать диалог сразу с несколькими пользователями, нам нужно также ввести некоторый идентификатор сессии `session_id`, и для каждой сессии помнить свою историю переписки и `id` последнего сообщения. По умолчанию будем использовать сессию `default`. 

In [8]:
exercise_db = {}
class Exercise(BaseModel):
    """Эта функция позволяет добавлять информацию о сделанном в зале упражнении."""

    тип: Optional[str] = Field(description="Тип упражнения (кардио или силовое)", default=None)
    название: Optional[str] = Field(description="Название упражения", default=None)
    болевые_ощущения : Optional[str] = Field(description="Болевые ощущения при выполнении упражнения", default=None)
    пульс : Optional[str] = Field(description="Пульс в момент выполнения упражнения", default=None)
    подходы : Optional[str] = Field(description="Количество подходов", default=None)
    повторения : Optional[str] = Field(description="Количество повторений", default=None)

    def process(self, session_id):
        if session_id not in exercise_db:
            exercise_db[session_id] = []
        exercise_db[session_id].append(self)
        return "Упражнение добавлено"

class ListExercises(BaseModel):
    """Эта функция позволяет получить список сделанных упражнений"""

    def process(self,session_id):
        if session_id not in exercise_db:
            return "Упражнений нет"
        else:
            return '\n'.join([
                f"{i+1}. {x.название} ({x.тип}, {x.подходы} подходов, {x.повторения} повторений)" for i,x in enumerate(exercise_db[session_id])
            ])


In [19]:
class Assistant():

    def __init__(self, instruction, tools = [], model=model,tool_choice='auto'):
        self.instruction = instruction
        self.model = model
        self.tool_choice = tool_choice
        self.tool_map = { x.__name__ : x for x in tools if issubclass(x, BaseModel) }
        self.tools = [
            self._create_tool_annot(x) for x in tools
        ]
        self.user_sessions = {}

    def _create_tool_annot(self, x):
        if issubclass(x, BaseModel):
            return {
                "type": "function",
                "name": x.__name__,
                "description": x.__doc__,
                "parameters": x.model_json_schema(),
            }
        else:
            return x

    def __call__(self, message, session_id='default'):
        s = self.user_sessions.get(session_id,{ 'previous_response_id' : None, 'history' : [] })
        s['history'].append({ 'role': 'user', 'content': message })
        res = client.responses.create(
            model = self.model,
            store = True,
            tools = self.tools,
            tool_choice = self.tool_choice,
            instructions = self.instruction,
            previous_response_id = s['previous_response_id'],
            input = message
        )
        # Обрабатываем вызов локальных инструментов
        tool_calls = [item for item in res.output if item.type == "function_call"]
        if tool_calls:
            s['history'].append({ 'role' : 'func_call', 'content' : res.output_text })
            out = []
            for call in tool_calls:
                args = call.arguments or "{}"
                print(f" + Обрабатываем: {call.name} ({call.arguments})")
                try:
                    fn = self.tool_map[call.name]
                    obj = fn.model_validate(json.loads(args))
                    result = obj.process(session_id)
                except Exception as e:
                    result = f"Ошибка: {e}"
                    print(f"Ошибка вызова: {e}")
                #print(f" + Результат: {result}")
                out.append({
                    "type": "function_call_output",
                    "call_id": call.call_id,
                    "output": result
                })
                res = client.responses.create(
                    model=self.model,
                    input=out,
                    tools=self.tools,
                    previous_response_id=res.id,
                    store=True
                )
        s['previous_response_id'] = res.id
        s['history'].append({ 'role' : 'assistant', 'content' : res.output_text })
        self.user_sessions[session_id]=s
        return res

    def history(self, session_id='default'):
        return self.user_sessions[session_id]['history']

In [20]:
instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Ты также можешь вести 
дневник выполненных пользователем упражнений - для этого используй функцию `Exercise`. Чтобы
показать список выполненных упражнений - используй `ListExercises`.
"""

assistant = Assistant(instruction, [Exercise, ListExercises])

printx(assistant('Привет! Посоветуй, как начать тренироваться в зале!').output_text)

Привет! Отлично, что решил начать тренироваться — это первый и самый важный шаг. Давай начнём с основ.

### 🔹 С чего начать?
1. **Определи цель**:
   - Набор мышечной массы?
   - Похудение и сжигание жира?
   - Улучшение выносливости и общего самочувствия?
   - Силовые достижения?

2. **Частота тренировок**:
   - Для новичков оптимально 3–4 раза в неделю.
   - Можно начать с **разделения по группам мышц**:
     - День 1: Грудь + Трицепс
     - День 2: Спина + Бицепс
     - День 3: Ноги + Плечи
   - Или с **полного тела (full-body)** 3 раза в неделю.

3. **Базовые упражнения**:
   - Приседания (с собственным весом или со штангой)
   - Жим лёжа
   - Тяга штанги/гантели в наклоне
   - Подтягивания (если можешь — если нет, начни с вспомогательных упражнений)
   - Жим гантелей над головой

4. **Подходы и повторения**:
   - Новичкам: 3–4 подхода по 8–12 повторений.
   - Отдых между подходами: 60–90 секунд.

5. **Разминка перед тренировкой** (5–10 минут):
   - Лёгкая кардионагрузка (ходьба на беговой дорожке, велотренажёр)
   - Динамическая растяжка
   - Лёгкие подходы перед основными упражнениями

6. **Питание**:
   - Белок: 1.5–2 г на кг веса тела (курица, рыба, яйца, творог, бобовые)
   - Углеводы: крупы, овощи, фрукты — особенно до и после тренировки
   - Жиры: орехи, авокадо, оливковое масло
   - Пей достаточно воды (2–3 литра в день)

---

Хочешь, я помогу тебе составить первую тренировку? Или расскажи, как прошла твоя первая тренировка — можем её зафиксировать и проанализировать.

In [21]:
printx(assistant('Я сделал 10 приседаний, запиши!').output_text)

 + Обрабатываем: Exercise ({"тип": "силовое", "название": "Приседания", "подходы": "1", "повторения": "10"})


Записал! Ты выполнил:  
✅ Приседания — 1 подход по 10 повторений, силовое упражнение.

Хочешь добавить ещё подходы или перейти к следующему упражнению? Например, отжимания или выпады — могу подсказать, как правильно их делать.

In [22]:
printx(assistant('Напомни, какие я сделал упражнения?').output_text)

 + Обрабатываем: ListExercises ()


Вот список упражнений, которые ты выполнил:

1. Приседания — силовое, 1 подход по 10 повторений  
2. Приседания — силовое, 1 подход по 10 повторений  
3. Приседания — силовое, 1 подход по 10 повторений  

Похоже, ты три раза записывал одни и те же приседания. Если это была одна серия из 3 подходов — лучше указывать количество подходов сразу. Например:  
"Сделал 3 подхода по 10 приседаний".

Хочешь, я помогу правильно оформить запись или предложу следующее упражнение для тренировки? 💪

## Добавляем RAG

Наш ассистент сам по себе неплохо отвечает на вопросы по тренировкам, однако если мы хотим сделать на основе бота консультанта конкретного фитнес-клуба, который должен обладать конкретными знаниями о его работе - необходимо дополнить его знания специализированной информацией. Для этого мы можем использовать RAG - подход, который позволяет искать релевантную информацию по запросу в текстовых базах знаний.

В качестве текстовой базы знаний в нашем примере возьмём несколько популярных текстов о фитнесе:

In [23]:
from glob import glob
import pandas as pd
import tiktoken

def get_tokenizer(model):
    try:
        return tiktoken.encoding_for_model(model)
    except:
        return tiktoken.encoding_for_model("gpt-5-nano") 

def get_token_count(filename):
    tokenizer = get_tokenizer(model)
    with open(filename, "r", encoding="utf8") as f:
        return len(tokenizer.encode(f.read()))

def get_file_len(filename):
    with open(filename, encoding="utf-8") as f:
        l = len(f.read())
    return l

d = [
    {
        "File": fn,
        "Tokens": get_token_count(fn),
        "Chars": get_file_len(fn),
    }
    for fn in glob("data/text-kb/*.txt")
]

df = pd.DataFrame(d)
df

,File,Tokens,Chars
0,data/text-kb\diary.txt,247,931
1,data/text-kb\how-to-begin.txt,811,3249
2,data/text-kb\program.txt,231,937
3,data/text-kb\qna.txt,1712,7098
4,data/text-kb\what-to-take.txt,262,954


В случае с RAG текстовая база знаний делится на небольшие фрагменты, по которым в ходе запроса осуществляется поиск. Оптимальная длина фрагмента определяется опытным путём, но в среднем разумно придерживаться размера 1000-2000 токенов на фрагмент. В нашем случае длина некоторых файлов превышает 1000 токенов, поэтому мы будем использовать **стратегию чанкования**, для разбиения текстовых файлов на более мелкие фрагементы.

## Загружаем файлы в облако

Чтобы RAG мог осущетвлять поиск по фрагментам файлов, нам необходимо построить индекс, а перед этим - загрузить все файлы в облако.

In [24]:
def upload_file(filename):
    return client.files.create(file=open(filename,'rb'),purpose='assistants')

df["Uploaded"] = df["File"].apply(upload_file)

## Строим векторный индекс

Для индексации файлов необходимо создать векторный индекс (vector store), и поместить все файлы туда:


In [25]:
vector_store = client.vector_stores.create(name='rag_store')

def add_to_store(file):
    client.vector_stores.files.create(
        vector_store_id=vector_store.id, 
        file_id=file.id,
        chunking_strategy={
            "type": "static",
            "static" : { "max_chunk_size_tokens" : 1000, "chunk_overlap_tokens" : 100 }
        }
        )

_ = df['Uploaded'].apply(add_to_store)

Теперь мы можем искать внутри нашего векторного хранилища:

In [26]:
res = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query="С чего начать занятия в зале?"
)
for x in res.data:
    print(f"{len(x.content[0].text)} символов из файла {x.filename}, релевантность = {x.score}")

3989 символов из файла qna.txt, релевантность = 0.9988905247000135
3264 символов из файла how-to-begin.txt, релевантность = 0.9154609745313269
941 символов из файла diary.txt, релевантность = 0.6090118099907961
947 символов из файла program.txt, релевантность = 0.33553025115033
3230 символов из файла qna.txt, релевантность = 0.17889788379920882


## Собираем RAG-ассистента

Для задействования RAG достаточно передать описание **инструмента поиска** в запрос Responses API:

In [27]:
search_tool = {
    "type" : "file_search",
    "vector_store_ids" : [vector_store.id]
}

res = client.responses.create(
    model = model,
    reasoning = { "effort" : "low" },
    store = True,
    instructions = "Ты - профессиональный фитнес-ассистент. Отвечай как энергичный молодой человек со спортивным задором",
    input = "С чего начать тренировки в зале?",
    tools = [search_tool]
)

res.to_dict()

{'id': '08b22582-e7d8-47c9-b410-8461e2201e76',
 'created_at': 1769696244.0,
 'error': None,
 'incomplete_details': None,
 'instructions': 'Ты - профессиональный фитнес-ассистент. Отвечай как энергичный молодой человек со спортивным задором',
 'metadata': {},
 'model': 'gpt://b1gkp4q6n092isf1ee3o/qwen3-235b-a22b-fp8/latest',
 'object': 'response',
 'output': [{'id': 'ffba2f0f-8e56-4698-961b-5b6eb3e12bb9',
   'queries': ['начало тренировок в зале для новичков'],
   'status': 'completed',
   'type': 'file_search_call',
   'results': [{'attributes': {},
     'file_id': 'fvt33ka3cljt75si23m4',
     'filename': 'qna.txt',
     'score': 0.9994098,
     'text': '### Если я хочу начать заниматься в спортзале, то с чего мне стоит начать?\r\n\r\nВ первую очередь нужно оценить состояние своего здоровья и поставить цель. От этого будет зависеть, в каком направлении вы будете работать: силовые тренировки, пилатес, кроссфит, функциональный тренинг или что-то другое. Выберите удобную одежду, которая н

Мы видим, что первым пунктом в поле `output` находится вызов инструмента поиска, с перефразированным запросом, а далее уже ответ ассистента, в котором в поле `annotations` содержатся ссылки на использованные фрагменты текста из базы знаний.

Мы может также передать инструмент поиска в созданный нами класс `Assistant`, чтобы использовать поиск совместно с другими инструментами:

In [28]:
search_tool = {
    "type" : "file_search",
    "vector_store_ids" : [vector_store.id],
    "max_num_results" : 5
}

instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Отвечай на основе имеющейся
дополнительной информации из файловой базы знаний, вызывая инструмент поиска `search_tool`.
Ты также можешь вести дневник выполненных пользователем упражнений - для этого используй 
функцию `Exercise`. Чтобы показать список выполненных упражнений - используй `ListExercises`.
"""

assistant = Assistant(instruction, [search_tool, Exercise, ListExercises],tool_choice='required')

res = assistant("Что нужно, чтобы начать заниматься в зале?")
printx(res.output_text)

Чтобы начать заниматься в зале, нужно учесть несколько важных аспектов:

### 1. **Оцените состояние здоровья и поставьте цель**
Перед началом тренировок важно понять, зачем вы идете в зал: похудеть, нарастить мышцы, улучшить выносливость или просто поддерживать форму. От цели зависит тип тренировок (силовые, кардио, функциональный тренинг и т.д.).  
Также рекомендуется проконсультироваться с врачом, особенно при наличии хронических заболеваний.

---

### 2. **Подготовьте экипировку**
- **Одежда**: удобная, дышащая, не сковывающая движения (спортивная майка, шорты или леггинсы).
- **Обувь**: кроссовки с хорошей амортизацией и устойчивостью. Для силовых тренировок лучше выбирать обувь с жесткой подошвой.
- **Аксессуары**:
  - Бутылка с водой (для поддержания водного баланса).
  - Полотенце (для вытирания пота и гигиены тренажеров).
  - Сменная обувь для душа и гигиенические принадлежности (шампунь, мыло).

---

### 3. **Пройдите вводный инструктаж**
В первый день в зале обязательно запишитесь на **бесплатный стартовый урок с тренером**. Он:
- Познакомит вас с оборудованием.
- Объяснит, как правильно пользоваться тренажерами.
- Научит правильной технике выполнения упражнений.
- Покажет расположение зон: кардио, силовая, раздевалки и т.д.

---

### 4. **Начните с разминки и умеренных нагрузок**
- Всегда начинайте тренировку с **10-минутной разминки** (суставная гимнастика, легкий бег на беговой дорожке).
- Начинайте с **небольших весов**, чтобы отработать технику.
- Делайте по **2–3 подхода по 8–12 повторений**, отдыхая 2–3 минуты между подходами.

---

### 5. **Не тренируйтесь в первый день «всё и сразу»**
- Не пытайтесь сделать максимум нагрузки — это может привести к перетренированности и травмам.
- Достаточно **2–3 тренировок в неделю** для начала.
- Дайте телу время адаптироваться.

---

### 6. **Обратите внимание на технику**
- **Техника важнее веса.** Неправильное выполнение упражнений может привести к травмам.
- Просите помощи у тренеров или опытных посетителей — в зале принято помогать новичкам.

---

### 7. **Следите за пульсом**
Используйте пульсометр, чтобы не перегружать сердце. Максимальный безопасный пульс можно рассчитать по формуле:  
**(220 – ваш возраст) × 0.7–0.9**

---

### 8. **Восстанавливайтесь и питайтесь правильно**
- Сон и питание — ключ к результату.
- Результат станет заметен уже через месяц, но значимые изменения — через 3 месяца при регулярных тренировках и правильном питании.

---

### 9. **Не бойтесь**
- Никто не осуждает новичков. Все начинали с нуля.
- Смело задавайте вопросы — вам всегда помогут.

---

### 10. **Рассмотрите возможность занятий с тренером**
Хотя бы на первые 1–2 месяца — это поможет избежать ошибок и закрепить правильную технику.

---

Если хочешь, могу помочь составить первую тренировку или записать её в дневник после посещения зала.

Посмотрим, из каких источников был получен этот ответ:

In [29]:
[ x for x in res.output ]

[ResponseFileSearchToolCall(id='80988981-3d96-4103-b84b-c5944a38f409', queries=['что нужно, чтобы начать заниматься в зале'], status='completed', type='file_search_call', results=[Result(attributes={}, file_id='fvt33ka3cljt75si23m4', filename='qna.txt', score=1.0, text='### Если я хочу начать заниматься в спортзале, то с чего мне стоит начать?\r\n\r\nВ первую очередь нужно оценить состояние своего здоровья и поставить цель. От этого будет зависеть, в каком направлении вы будете работать: силовые тренировки, пилатес, кроссфит, функциональный тренинг или что-то другое. Выберите удобную одежду, которая не сковывает движения, и кроссовки с амортизацией. Также стоит приобрести пульсометр, чтобы контролировать свой ритм и не перегружать сердце во время упражнений. Для каждого возраста существует свой максимальный диапазон частоты сердечных сокращений. Его можно рассчитать по формуле: (220- свой возраст) х 70%/80%/90%. Максимально допустимая частота для безопасного тренинга – 90%.\r\n\r\n\r\n

In [30]:
from openai.types.responses.response_output_message import ResponseOutputMessage

def find_obj(t,l):
    for x in l:
        if isinstance(x,t):
            return x
    return None

def print_citations(result):
    o = find_obj(ResponseOutputMessage,result.output)
    for x in o.content[0].annotations:
        if x.type == "file_citation":
            print(f"{x.filename}, idx={x.index}")

print_citations(res)

qna.txt, idx=0
how-to-begin.txt, idx=0
program.txt, idx=0
qna.txt, idx=0
what-to-take.txt, idx=0


## Добавляем таблицу добавок

Для серьезных тренировок важны также пищевые добавки. Чтобы добавить информацию о них в нашего ассистента, мы нашли таблицу таких добавок в формате markdown:

In [31]:
with open("data/additives.md", encoding="utf-8") as f:
    additives = f.readlines()
additives_all = "".join(additives)

tokenizer = get_tokenizer(model)
tokens = len(tokenizer.encode(additives_all))
print(f"Токенов: {tokens}, {len(additives_all)/tokens} chars/token")

Токенов: 6833, 4.82218644811942 chars/token


In [32]:
printx(additives_all[:1040])

| Название добавки                                 | Категория                                      | Назначение                                                                                                 | Доза                          | Рейтинг |
|--------------------------------------------------|-----------------------------------------------|-----------------------------------------------------------------------------------------------------------|-------------------------------|---------|
| Креатин                                          | Для наращивания мышечной массы, силы и ускорения восстановления | Увеличивает физическую силу, ускоряет восстановление; увеличивает массу                                   | 2-20 г в день                 | *****   |
| Глютамин                                         | Для наращивания мышечной массы, силы и ускорения восстановления | Предотвращает распад мышечной ткани; укрепляет иммунную систему                                           | 5-20 г в день                 | *****   

Видим, что табличка большая, поэтому её придётся *чанковать*. Но при этом важно чанковать табличку так, чтобы в каждом фрагмента оставался заголовок таблицы, который определяет семантику столбцов.

Отделим заголовок таблицы:

In [33]:
header = additives[:2]
header

['| Название добавки                                 | Категория                                      | Назначение                                                                                                 | Доза                          | Рейтинг |\n',
 '|--------------------------------------------------|-----------------------------------------------|-----------------------------------------------------------------------------------------------------------|-------------------------------|---------|\n']

Ниже будем чанковать табличку вручную, задав размер чанка в символах для простоты. Мы будем сразу загружать получившиеся фрагменты в облако, минуя диск:

In [34]:
import io

chunk_size = 600 * 5  # около 600 tokens * 5 char/token

s = header.copy()
uploaded_additives = []
i = 0
for x in additives[2:]:
    s.append(x)
    if len("".join(s)) > chunk_size:
        f = client.files.create(
            purpose="assistants",
            file = (f'table_{i}.txt',io.BytesIO("".join(s).encode("utf-8")),'text/markdown')
        )
        client.vector_stores.files.create(file_id=f.id, vector_store_id=vector_store.id)
        uploaded_additives.append(f)
        i+=1
        s = header.copy()
print(f"Uploaded {len(uploaded_additives)} table chunks")

Uploaded 12 table chunks


Посмотрим, как система стала отвечать на вопросы о добавках:

In [62]:
res = assistant("Какие добавки помогают нарастить мышечную массу?")

printx(res.output_text)
print_citations(res)

Для наращивания мышечной массы наиболее эффективными и популярными спортивными добавками являются:

### 1. **Креатин**  
- **Назначение**: Увеличивает физическую силу, ускоряет восстановление, способствует росту мышечной массы.  
- **Доза**: 2–20 г в день (обычно 3–5 г в день достаточно для поддержания эффекта).  
- **Рейтинг**: ⭐⭐⭐⭐⭐  
- **Примечание**: Одна из самых изученных и безопасных добавок. Рекомендована для силовых тренировок.

### 2. **Белковые добавки (протеин)**  
- **Виды**: Сывороточный, казеиновый, яичный, соевый, говяжий, конопляный протеин.  
- **Назначение**: Источник аминокислот для построения мышечной ткани, особенно важен после тренировки.  
- **Доза**: 20–40 г после тренировки, в зависимости от веса и целей.  
- **Рейтинг**: ⭐⭐⭐⭐⭐  
- **Примечание**: Идеально подходит, если вы не можете получить достаточно белка с пищей.

### 3. **BCAA (аминокислоты с разветвленной цепью)**  
- **Назначение**: Предотвращают катаболизм (распад мышц), ускоряют восстановление.  
- **Доза**: 5–10 г до или после тренировки.  
- **Рейтинг**: ⭐⭐⭐⭐⭐  
- **Примечание**: Особенно полезны при интенсивных тренировках или сушке.

### 4. **Глютамин**  
- **Назначение**: Уменьшает распад мышечной ткани, укрепляет иммунитет, ускоряет восстановление.  
- **Доза**: 5–10 г в день (часто после тренировки и перед сном).  
- **Рейтинг**: ⭐⭐⭐⭐⭐  

### 5. **Гейнер**  
- **Назначение**: Помогает набрать массу при недостатке калорий. Содержит белок и углеводы.  
- **Доза**: 50–100 г после тренировки или между приёмами пищи.  
- **Примечание**: Подходит худощавым людям («хардгейнерам»).

### 6. **L-Орнитин и L-Аргинин**  
- **Назначение**: Стимулируют выработку гормона роста, улучшают приток крови к мышцам.  
- **Доза (Орнитин)**: 5–10 г в день.  
- **Рейтинг**: ⭐⭐  

### 7. **Конопляное и льняное масло**  
- **Назначение**: Источники незаменимых жирных кислот, укрепляют сердечно-сосудистую систему, поддерживают гормональный фон.  
- **Доза**: 1–2 столовые ложки в день.  
- **Рейтинг**: ⭐⭐⭐⭐⭐  

---

### Советы по применению:
- **Основа — питание и тренировки**. Добавки работают только в сочетании с правильным питанием и нагрузкой.
- **Протеин и креатин** — базовые и безопасные добавки для новичков.
- **Гейнер** — если не получается набирать вес.
- **BCAA и глютамин** — при высоких нагрузках.
- **Избегайте "анаболиков" и гормонов** — они опасны без медицинских показаний.

💡 **Важно**: Перед началом приёма любых добавок проконсультируйтесь с врачом, особенно если есть хронические заболевания.

table_0.txt, idx=0
table_7.txt, idx=0
table_1.txt, idx=0
table_2.txt, idx=0
table_8.txt, idx=0
table_6.txt, idx=0
table_6.txt, idx=0
table_4.txt, idx=0
qna.txt, idx=0
table_5.txt, idx=0


## Добавляем веб-поиск

Очень частый запрос к фитнесс-ассистентам - сколько стоит годовой абонемент в фитнес-клуб? Наш ассистент сможет ответить на этот вопрос:

In [35]:
res = assistant("Сколько стоит годовой абонемент в фитнес-клуб в Санкт-Петербурге?")
printx(res.output_text)

К сожалению, в предоставленном контексте **не указана конкретная стоимость годового абонемента в фитнес-клуб в Санкт-Петербурге**. Упоминается лишь, что **вводный инструктаж входит в стоимость абонемента**, но сама цена не названа.

Стоимость абонементов может сильно варьироваться в зависимости от сети фитнес-клубов, уровня оснащения, расположения и дополнительных услуг (например, групповые занятия, бассейн, тренерские консультации).

Ориентировочные цены в Санкт-Петербурге (по общему рынку на 2024–2025 годы):

- **Бюджетные клубы (например, "Спортмастер Фитнес", "Карнавал")**: от 15 000 до 30 000 рублей в год.
- **Средний сегмент (например, "Атлетика", "World Class Lite")**: от 30 000 до 60 000 рублей в год.
- **Премиум-клубы (например, "World Class", "X-Fit Black", "Живи!")**: от 60 000 до 150 000 рублей и выше в год.

Рекомендую посетить сайты популярных фитнес-сетей в Санкт-Петербурге или прийти в зал лично, чтобы уточнить актуальные тарифы и возможные скидки для новичков.

Однако, данные эти данные основаны на внутренней информации модели, которая могла устареть с момента последнего обновления. Кроме того, велика вероятность галлюцинации. Исправить эти проблемы можно с помощью добавления веб-поиска.

Инструмент веб-поиска в простейшем варианты задаётся простым и коротким описанием: 

In [36]:
web_search_tool = {
    "type" : "web_search"
}

instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Отвечай на основе имеющейся
дополнительной информации из файловой базы знаний, вызывая инструмент поиска `search_tool`. В случае, если запрос касается упоминания абстрактных фитнес-клубов или
новостей - используй поиск в интернет `web_search_tool`.
Ты также можешь вести дневник выполненных пользователем упражнений - для этого используй 
функцию `Exercise`. Чтобы показать список выполненных упражнений - используй `ListExercises`.
"""

assistant = Assistant(instruction, [web_search_tool, search_tool, Exercise, ListExercises],tool_choice='required')

res = assistant("Сколько стоит годовой абонемент в фитнес-клуб в Санкт-Петербурге?")
printx(res.output_text)

Стоимость годового абонемента в фитнес-клуб в Санкт-Петербурге зависит от уровня клуба, его расположения, наличия дополнительных услуг (бассейн, сауна, групповые занятия и т.д.).

На основе доступной информации:

- **В недорогих или средних фитнес-клубах** (например, Spirit Fitness, Praktik, ВСЕМФИТНЕС):  
  Годовой абонемент может стоить от **20 000 до 40 000 рублей**.  
  Например:
  - В сети **Spirit Fitness** месячный абонемент от 1700 ₽ — годовая подписка составит около **20 400 ₽**.
  - В некоторых клубах, например **Praktik**, есть предложения по **350 ₽ в месяц** за абонемент на выходные — это около **4 200 ₽ в год**, но с ограничениями по времени.

- **В премиальных фитнес-клубах** (например, A-Fitness, Fitness House PRESTIGE, World Class):  
  Цены значительно выше — от **50 000 до 100 000 рублей и более** за год.  
  Например:
  - На Avito продаётся абонемент в **Crocus Fitness** за **65 000 ₽** (безлимитный, с бассейном).
  - В **A-Fitness** аналогичные абонементы стоят от **75 000 до 85 000 ₽** в год.

### Рекомендации:
- Если вы ищете **экономичный вариант** — обратите внимание на локальные клубы, акции, студенческие тарифы или тарифы по предоплате.
- Для **комфортных условий** (бассейн, сауна, много тренажёров) — средний ценовой сегмент (30–50 тыс. ₽/год).
- Если важны **премиум-услуги и сервис** — готовьте бюджет от 60 тыс. ₽ и выше.

Хочешь — могу помочь подобрать оптимальный клуб под твои цели и бюджет.

Чтобы убедиться, что инструмент был вызван - посмотрим на длину объекта `res.output`:

In [40]:
len(res.output)

2

Первый элемент соответствует проделанному поисковому запросу, и из него можно извлечь сам текст запроса, который LLM решила искать в интернет:

In [42]:
res.output[0].action.query

'{"query": "цена годового абонемента в фитнес-клуб Санкт-Петербург", "lang": "ru"}'

Второй элемент - это текстовый ответ модели. Также как и в случае RAG, в нём содержится список использованных источников `annotations`, благодаря которому мы можем посмотреть на то, какие именно сайты использовались при формировании ответа:

In [43]:
for x in res.output[1].content[0].annotations:
    print(f"{x.title} - {x.url}")

Абонементы фитнес-клуба Spirit Fitness в Санкт-Петербурге (СПб): абонементы от 1700 ₽ в месяц - https://spiritfit.ru/spb/abonement/
Тренажерные залы: цены от 11 руб. в Санкт-Петербурге – Рейтинг спортзалов – Zoon.ru - https://zoon.ru/spb/fitness/price/trenazhernyj_zal/
абонемент в фитнес клуб год - Авито | Объявления в Санкт-Петербурге: купить вещь, выбрать исполнителя или работу, подобрать недвижимость и транспорт по низкой цене | Авито - https://www.avito.ru/sankt-peterburg?q=%D0%B0%D0%B1%D0%BE%D0%BD%D0%B5%D0%BC%D0%B5%D0%BD%D1%82+%D0%B2+%D1%84%D0%B8%D1%82%D0%BD%D0%B5%D1%81+%D0%BA%D0%BB%D1%83%D0%B1+%D0%B3%D0%BE%D0%B4


Как правило, нас интересует стоимость фитнеса в конкретном городе, где мы живём. Мы можем указать город в запросе, но существует и другая возможность - ограничить поиск только определённой территорией или доменом.

Инструмент веб-поиска допускает следующие параметры (подробно описаны в [документации](https://yandex.cloud/ru/docs/ai-studio/concepts/agents/tools/websearch)):

* **filters** - устанавливает ограничение на область поиска. Можно ограничивать поиск по списку доментов (**allowed_domains**), или по региону (**user_location**). Список возможных регионов доступен [здесь](https://yandex.cloud/ru/docs/search-api/reference/regions).
* **search_context_size** - позволяет указать размер контекста, получаемого LLM из поиска. Возможные значения: **low**, **medium** (используется по умолчанию) и **high**. Более подробный контекст ведёт к повышению потребления токенов, но и позволяет получить более правильный и детальный ответ.

Вот пример описания инструмента поиска с заданием региона и доступных доменов:

In [ ]:
web_search_tool = {
    "type": "web_search",
    "filters": {
        "allowed_domains": [
            "otzovik.com"
        ],
        "user_location": {
            "region": "213", # Москва
        },
    },
    "search_context_size": "medium", # варианты: low | medium | high
}

## Удаляем лишнее

В заключение удалим созданные ресурсы!

> **ВНИМАНИЕ**: Не выполняйте этот код, если у вас есть другие файлы или поисковые индексы в проекте!  

In [72]:
vector_stores = client.vector_stores.list()
for v in vector_stores:
    print(f" + Deleting vector store id={v.id}")
    client.vector_stores.delete(vector_store_id=v.id)

files = client.files.list(purpose='assistants')
for f in files:
    print(f" + Deleting file id={f.id}")
    client.files.delete(file_id=f.id)

 + Deleting vector store id=vs_68c05ae6dfb48191ba2d825dfb4b40dc
 + Deleting vector store id=vs_68c05a9d567c8191930aa77680d6ccd6
 + Deleting vector store id=vs_68c05a173d048191b72312f9d50ef54c
 + Deleting vector store id=vs_68c058ec59a08191ba058f6e0e1d8bd6
 + Deleting file id=file-HRiSxGaE7VfNMk2CzfZtZo
 + Deleting file id=file-MjjbkkC5HD9HRSmQSQQpT7
 + Deleting file id=file-ESK3JQ5gWQMMby62U6piyG
 + Deleting file id=file-GtR8YNeAA1StqhzKrnkGRc
 + Deleting file id=file-TXHnQP2NT8x9XP4n4tuFjw
 + Deleting file id=file-LbhNajaWMZaPQEF28zR8cf
 + Deleting file id=file-Y4ChZMU95S9V8J6vumzgr3
 + Deleting file id=file-WeyrdfmPUyKT2puZLz4vc3
 + Deleting file id=file-L1mLyVH3qXzaCnihAoB7gh
 + Deleting file id=file-YQSFygEJCk2p2AvC9Yjcgr
 + Deleting file id=file-4s7d9LZg2fCH1djtmt1pR4
 + Deleting file id=file-6R1Kw713mi7N4v9WQfv7SC
 + Deleting file id=file-AKvcrc2ke4HcEN913TRLwk
 + Deleting file id=file-NuFMW2wowg1XK9cvytKeZr
 + Deleting file id=file-QMpTxUm79v95dgARH8zhYh
 + Deleting file id=file